In [43]:
from mako.template import Template

import argparse
import configparser
from itertools import chain
import re

from sys import platform as _platform
import os
import functools

In [44]:
DEBUG = True
IMAGE_NAME = 'quagga'
LINUX_TERMINAL_TYPE = 'xterm'
PATH_TO_LAB = 'shared/MARACAS_lab/'

In [45]:
if _platform == "linux" or _platform == "linux2":
    print ('linux')
elif _platform == "darwin":
    print ('MAC OS X')
elif _platform == "win32":
    print ('Windows')

Windows


# Parsing of lab.conf

In [46]:
config = configparser.ConfigParser()

In [47]:
# reads lab.conf
with open(PATH_TO_LAB + 'lab.conf') as stream:
    # adds a section to mimic a .ini file
    stream = chain(("[dummysection]",), stream)
    config.read_file(stream)

In [48]:
# gets 2 list of keys, one for machines and the other for the metadata
# we also need a unique list of links
keys = []
m_keys = []
links = []
for key in config['dummysection']: 
    if DEBUG: print(key, config['dummysection'][key])
    if '[' in key and ']' in key:
        keys.append(key)
        links.append(config['dummysection'][key])
    else:
        m_keys.append(key)

lab_description "absaaaaaaaaaaaaaaaaa"
lab_version "1.0"
lab_author "aaaaaaaaaaaaaaaa"
lab_email "aaaaaaaaa@aaaaaaaaaaaaaa"
host[0] A
server[0] B
ospf1[0] C
ospf1[1] E
ospf1[2] F
ospf2[0] D
ospf2[1] E
ospf3[0] A
ospf3[1] D
ospf3[2] C
br[0] G
br[1] F
rip1[0] H
rip1[1] G
rip2[0] B
rip2[1] H


In [49]:
config['dummysection'][keys[0]]

'A'

In [50]:
# we only need unique links
links = set(links)

In [51]:
links

{'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'}

In [52]:
# helper functions for natural sorting
def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

# sort the keys so that we keep the order of the interfaces
keys.sort(key=natural_keys)

In [53]:
keys

['br[0]',
 'br[1]',
 'host[0]',
 'ospf1[0]',
 'ospf1[1]',
 'ospf1[2]',
 'ospf2[0]',
 'ospf2[1]',
 'ospf3[0]',
 'ospf3[1]',
 'ospf3[2]',
 'rip1[0]',
 'rip1[1]',
 'rip2[0]',
 'rip2[1]',
 'server[0]']

In [54]:
m_keys

['lab_description', 'lab_version', 'lab_author', 'lab_email']

In [55]:
# we then get a dictionary of machines ignoring interfaces that have missing positions (eg: 1,3,6 instead of 0,1,2)
machines = {}
for key in keys: 
    splitted = key.split('[')
    name = splitted[0]
    splitted = splitted[1].split(']')
    ifnumber = int(splitted[0])
    if not machines.get(name):
        machines[name] = []
    if len(machines[name]) == 0 or machines[name][len(machines[name])-1][1] == ifnumber - 1:
        machines[name].append((config['dummysection'][key], ifnumber))

In [56]:
machines

{'br': [('G', 0), ('F', 1)],
 'host': [('A', 0)],
 'ospf1': [('C', 0), ('E', 1), ('F', 2)],
 'ospf2': [('D', 0), ('E', 1)],
 'ospf3': [('A', 0), ('D', 1), ('C', 2)],
 'rip1': [('H', 0), ('G', 1)],
 'rip2': [('B', 0), ('H', 1)],
 'server': [('B', 0)]}

In [57]:
metadata = {}
for m_key in m_keys:
    if config['dummysection'][m_key].startswith('"') and config['dummysection'][m_key].endswith('"'):
        config['dummysection'][m_key] = config['dummysection'][m_key][1:-1]
    metadata[m_key] = config['dummysection'][m_key]

In [58]:
metadata

{'lab_author': 'aaaaaaaaaaaaaaaa',
 'lab_description': 'absaaaaaaaaaaaaaaaaa',
 'lab_email': 'aaaaaaaaa@aaaaaaaaaaaaaa',
 'lab_version': '1.0'}

# Generation of docker-compose.yml (may not be used in the end)

In [59]:
network_block_template = '''networks: 
${links}
'''
network_link_template = '''    ${link}:
'''
service_block_template = '''version: '2.1'

services:
${services}
'''
service_template = '''    ${machine_name}:
        image: ${image_name}
        privileged: true
        container_name: netkit-${machine_name}
        ports:
            - 808${number}:80
        networks:
${links}
'''
service_link_template = '''            ${link}:
'''
# TODO lab.dep with '''depends_on: service: condition: service_healty'''

In [60]:
print(Template('''hello ${data}! 
               ''').render(data="world"))

hello world! 
               


In [61]:
conf_network = ""
for link in links:
    conf_network = conf_network + Template(network_link_template).render(link=link)

conf_network = Template(network_block_template).render(links=conf_network)

In [62]:
print(conf_network)

networks: 
    D:
    F:
    H:
    G:
    B:
    C:
    A:
    E:




In [63]:
conf_services = ""
counter = 0
for machine_name, interfaces in machines.items():
    conf_links = ""
    for link, _ in interfaces:
        conf_links = conf_links + Template(service_link_template).render(link=link)
    conf_services = conf_services + Template(service_template).render(machine_name=machine_name, image_name=IMAGE_NAME, number=counter, links=conf_links)
    counter += 1
    
conf_services = Template(service_block_template).render(services=conf_services)

In [64]:
print(conf_services)

version: '2.1'

services:
    rip2:
        image: quagga
        privileged: true
        container_name: netkit-rip2
        ports:
            - 8080:80
        networks:
            B:
            H:

    host:
        image: quagga
        privileged: true
        container_name: netkit-host
        ports:
            - 8081:80
        networks:
            A:

    server:
        image: quagga
        privileged: true
        container_name: netkit-server
        ports:
            - 8082:80
        networks:
            B:

    ospf2:
        image: quagga
        privileged: true
        container_name: netkit-ospf2
        ports:
            - 8083:80
        networks:
            D:
            E:

    br:
        image: quagga
        privileged: true
        container_name: netkit-br
        ports:
            - 8084:80
        networks:
            G:
            F:

    rip1:
        image: quagga
        privileged: true
        container_name: netkit-rip1
        ports:

# Generation of docker run and docker network commands (lstart)

In [65]:
create_network_template = 'docker network create '
create_network_commands = []
for link in links:
    create_network_commands.append(create_network_template + link)

In [66]:
create_machine_template = 'docker run -d --privileged --name {machine_name} -p 808{number}:80 --network={first_link} {image_name}'
# we could use -ti -a stdin -a stdout and then /bin/bash -c "commands;bash", 
# but that woult execute commands like ifconfig BEFORE all the networks are linked
create_machine_commands = []

create_connection_template = 'docker network connect {link} {machine_name}'
create_connection_commands = []

copy_folder_template = 'docker cp ' + PATH_TO_LAB + '{machine_name}/etc {machine_name}:/'
copy_folder_commands = []

exec_template = 'docker exec {params} -ti --privileged=true {machine_name} {command}'
exec_commands = []

count = 0
def replace_multiple_items(repls, string):
    return functools.reduce(lambda a, kv: a.replace(*kv), repls, string)

for machine_name, interfaces in machines.items():
    repls = ('{machine_name}', machine_name), ('{number}', str(count)), ('{first_link}', interfaces[0][0]), ('{image_name}', IMAGE_NAME)
    create_machine_commands.append(replace_multiple_items(repls, create_machine_template))
    count += 1
    for link,_ in interfaces[1:]:
        repls = ('{link}', link), ('{machine_name}', machine_name)
        create_connection_commands.append(replace_multiple_items(repls, create_connection_template))
    repls = ('{machine_name}', machine_name), ('{machine_name}', machine_name)
    copy_folder_commands.append(replace_multiple_items(repls, copy_folder_template))
    repls = ('{machine_name}', machine_name), ('{command}', 'bash  -c "echo -ne \'\\033]0;' + machine_name + '\\007\'; bash"'), ('{params}', '-e TERM=debian')
    exec_commands.append(replace_multiple_items(repls, exec_template))
    
    
# for each machine we have to get the machine.startup file and insert every non empty line as a string inside an array
startup_commands = []
for machine_name, _ in machines.items():
    f = open(PATH_TO_LAB + machine_name + '.startup', 'r')
    for line in f:
        if line.strip() and line not in ['\n', '\r\n']:
            repls = ('{machine_name}', machine_name), ('{command}', line.strip()), ('{params}', '-d')
            startup_commands.append(replace_multiple_items(repls, exec_template))
    f.close()

In [67]:
print (create_network_commands)
print (create_machine_commands)
print (create_connection_commands)
print (copy_folder_commands)

['docker network create D', 'docker network create F', 'docker network create H', 'docker network create G', 'docker network create B', 'docker network create C', 'docker network create A', 'docker network create E']
['docker run -d --privileged --name rip2 -p 8080:80 --network=B quagga', 'docker run -d --privileged --name host -p 8081:80 --network=A quagga', 'docker run -d --privileged --name server -p 8082:80 --network=B quagga', 'docker run -d --privileged --name ospf2 -p 8083:80 --network=D quagga', 'docker run -d --privileged --name br -p 8084:80 --network=G quagga', 'docker run -d --privileged --name rip1 -p 8085:80 --network=H quagga', 'docker run -d --privileged --name ospf1 -p 8086:80 --network=C quagga', 'docker run -d --privileged --name ospf3 -p 8087:80 --network=A quagga']
['docker network connect H rip2', 'docker network connect E ospf2', 'docker network connect F br', 'docker network connect G rip1', 'docker network connect E ospf1', 'docker network connect F ospf1', 'do

In [68]:
commands = create_network_commands + create_machine_commands + create_connection_commands + copy_folder_commands

In [69]:
print (commands)

['docker network create D', 'docker network create F', 'docker network create H', 'docker network create G', 'docker network create B', 'docker network create C', 'docker network create A', 'docker network create E', 'docker run -d --privileged --name rip2 -p 8080:80 --network=B quagga', 'docker run -d --privileged --name host -p 8081:80 --network=A quagga', 'docker run -d --privileged --name server -p 8082:80 --network=B quagga', 'docker run -d --privileged --name ospf2 -p 8083:80 --network=D quagga', 'docker run -d --privileged --name br -p 8084:80 --network=G quagga', 'docker run -d --privileged --name rip1 -p 8085:80 --network=H quagga', 'docker run -d --privileged --name ospf1 -p 8086:80 --network=C quagga', 'docker run -d --privileged --name ospf3 -p 8087:80 --network=A quagga', 'docker network connect H rip2', 'docker network connect E ospf2', 'docker network connect F br', 'docker network connect G rip1', 'docker network connect E ospf1', 'docker network connect F ospf1', 'dock

In [70]:
print (startup_commands)

['docker exec -d -ti --privileged=true rip2 ifconfig eth0 2.0.0.1/24 up', 'docker exec -d -ti --privileged=true rip2 ifconfig eth1 11.0.0.2/30 up', 'docker exec -d -ti --privileged=true rip2 /etc/init.d/zebra start', 'docker exec -d -ti --privileged=true host ifconfig eth0 1.0.0.2/24 up', 'docker exec -d -ti --privileged=true host route add default gw 1.0.0.1', 'docker exec -d -ti --privileged=true server ifconfig eth0 2.0.0.2/24 up', 'docker exec -d -ti --privileged=true server route add default gw 2.0.0.1', 'docker exec -d -ti --privileged=true server /etc/init.d/apache2 start', 'docker exec -d -ti --privileged=true ospf2 ifconfig eth0 10.0.0.6/30 up', 'docker exec -d -ti --privileged=true ospf2 ifconfig eth1 10.0.0.9/30 up', 'docker exec -d -ti --privileged=true ospf2 /etc/init.d/zebra start', 'docker exec -d -ti --privileged=true br ifconfig eth0 11.1.0.1/30 up', 'docker exec -d -ti --privileged=true br ifconfig eth1 10.1.0.1/30 up', 'docker exec -d -ti --privileged=true br /etc/in

In [71]:
print (exec_commands)

['docker exec -e TERM=debian -ti --privileged=true rip2 bash  -c "echo -ne \'\\033]0;rip2\\007\'; bash"', 'docker exec -e TERM=debian -ti --privileged=true host bash  -c "echo -ne \'\\033]0;host\\007\'; bash"', 'docker exec -e TERM=debian -ti --privileged=true server bash  -c "echo -ne \'\\033]0;server\\007\'; bash"', 'docker exec -e TERM=debian -ti --privileged=true ospf2 bash  -c "echo -ne \'\\033]0;ospf2\\007\'; bash"', 'docker exec -e TERM=debian -ti --privileged=true br bash  -c "echo -ne \'\\033]0;br\\007\'; bash"', 'docker exec -e TERM=debian -ti --privileged=true rip1 bash  -c "echo -ne \'\\033]0;rip1\\007\'; bash"', 'docker exec -e TERM=debian -ti --privileged=true ospf1 bash  -c "echo -ne \'\\033]0;ospf1\\007\'; bash"', 'docker exec -e TERM=debian -ti --privileged=true ospf3 bash  -c "echo -ne \'\\033]0;ospf3\\007\'; bash"']


# Run command to create lab

In [72]:
import subprocess
def run_command_detatched(cmd_line):
    p = subprocess.Popen(cmd_line, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    out = p.communicate()[0]
    return out

In [73]:
print (run_command_detatched('ping www.google.com & ping www.google.com'))

b'\r\nEsecuzione di Ping www.google.com [216.58.209.68] con 32 byte di dati:\r\nRisposta da 216.58.209.68: byte=32 durata=36ms TTL=54\r\nRisposta da 216.58.209.68: byte=32 durata=37ms TTL=54\r\nRisposta da 216.58.209.68: byte=32 durata=36ms TTL=54\r\nRisposta da 216.58.209.68: byte=32 durata=38ms TTL=54\r\n\r\nStatistiche Ping per 216.58.209.68:\r\n    Pacchetti: Trasmessi = 4, Ricevuti = 4, \r\n    Persi = 0 (0% persi),\r\nTempo approssimativo percorsi andata/ritorno in millisecondi:\r\n    Minimo = 36ms, Massimo =  38ms, Medio =  36ms\r\n\r\nEsecuzione di Ping www.google.com [216.58.209.68] con 32 byte di dati:\r\nRisposta da 216.58.209.68: byte=32 durata=47ms TTL=54\r\nRisposta da 216.58.209.68: byte=32 durata=38ms TTL=54\r\nRisposta da 216.58.209.68: byte=32 durata=46ms TTL=54\r\nRisposta da 216.58.209.68: byte=32 durata=44ms TTL=54\r\n\r\nStatistiche Ping per 216.58.209.68:\r\n    Pacchetti: Trasmessi = 4, Ricevuti = 4, \r\n    Persi = 0 (0% persi),\r\nTempo approssimativo percors

In [74]:
separator_unix = ' ; '
separator_windows = ' & '

open_bash_separator = ''

if _platform == "win32":
    open_bash_separator = separator_windows
elif _platform == "darwin":
    open_bash_separator = separator_unix
else: #linux or linux2
    open_bash_separator = separator_unix

In [75]:
lab_create_command_string = ''
for command in commands:
    lab_create_command_string += command + open_bash_separator
lab_create_command_string = lab_create_command_string[:len(lab_create_command_string)-2]
if DEBUG: print (lab_create_command_string)

docker network create D & docker network create F & docker network create H & docker network create G & docker network create B & docker network create C & docker network create A & docker network create E & docker run -d --privileged --name rip2 -p 8080:80 --network=B quagga & docker run -d --privileged --name host -p 8081:80 --network=A quagga & docker run -d --privileged --name server -p 8082:80 --network=B quagga & docker run -d --privileged --name ospf2 -p 8083:80 --network=D quagga & docker run -d --privileged --name br -p 8084:80 --network=G quagga & docker run -d --privileged --name rip1 -p 8085:80 --network=H quagga & docker run -d --privileged --name ospf1 -p 8086:80 --network=C quagga & docker run -d --privileged --name ospf3 -p 8087:80 --network=A quagga & docker network connect H rip2 & docker network connect E ospf2 & docker network connect F br & docker network connect G rip1 & docker network connect E ospf1 & docker network connect F ospf1 & docker network connect D osp

In [76]:
print (run_command_detatched(lab_create_command_string))

b'Error response from daemon: network with name D already exists\nError response from daemon: network with name F already exists\nError response from daemon: network with name H already exists\nError response from daemon: network with name G already exists\nError response from daemon: network with name B already exists\nError response from daemon: network with name C already exists\nError response from daemon: network with name A already exists\nError response from daemon: network with name E already exists\ndocker: Error response from daemon: Conflict. The container name "/rip2" is already in use by container d0b79d603902486ad372165ee2086851ca84601ee3702174ff8edcf012ece65b. You have to remove (or rename) that container to be able to reuse that name..\nSee \'docker run --help\'.\ndocker: Error response from daemon: Conflict. The container name "/host" is already in use by container 8cf505c377e1896c0ea88b38f259ac20501ac9acd3ebd060ffc4a86eab5e3c2f. You have to remove (or rename) that con

# Run startup commands inside lab and attach to lab

In [77]:
os.system('start /wait cmd /c "ping www.google.com & cmd.exe"')

0

In [78]:
os.system(LINUX_TERMINAL_TYPE + ' -e \'bash -c "ping www.google.com; exec bash"\'')

1

In [79]:
os.system('Terminal.app -c "ping www.google.com; exec Terminal.app"') # not tested yet

1

In [80]:
open_bash_template_linux = LINUX_TERMINAL_TYPE + ' -e \'bash -c "{commands}"\''
open_bash_template_windows = 'start cmd /k "{commands}"'
open_bash_template_osx = 'Terminal.app -e \'bash -c "{commands}"\''  # not tested yet

open_bash_template = ''

if _platform == "win32":
    open_bash_template = open_bash_template_windows
elif _platform == "darwin":
    open_bash_template = open_bash_template_osx
else: #linux or linux2
    open_bash_template = open_bash_template_linux

In [81]:
for startup_command in startup_commands:
    print (run_command_detatched(startup_command))

b''
b''
b'Error response from daemon: rpc error: code = 2 desc = oci runtime error: exec failed: container_linux.go:247: starting container process caused "exec: \\"/etc/init.d/zebra\\": stat /etc/init.d/zebra: no such file or directory"\n'
b''
b''
b''
b''
b'Error response from daemon: rpc error: code = 2 desc = oci runtime error: exec failed: container_linux.go:247: starting container process caused "exec: \\"/etc/init.d/apache2\\": stat /etc/init.d/apache2: no such file or directory"\n'
b''
b''
b'Error response from daemon: rpc error: code = 2 desc = oci runtime error: exec failed: container_linux.go:247: starting container process caused "exec: \\"/etc/init.d/zebra\\": stat /etc/init.d/zebra: no such file or directory"\n'
b''
b''
b'Error response from daemon: rpc error: code = 2 desc = oci runtime error: exec failed: container_linux.go:247: starting container process caused "exec: \\"/etc/init.d/zebra\\": stat /etc/init.d/zebra: no such file or directory"\n'
b''
b''
b'Error response

In [82]:
command_string = ''
for command in exec_commands:
    command_string += open_bash_template.replace('{commands}', command) + open_bash_separator
command_string = command_string[:len(command_string)-2]
if DEBUG: print (command_string)

start cmd /k "docker exec -e TERM=debian -ti --privileged=true rip2 bash  -c "echo -ne '\033]0;rip2\007'; bash"" & start cmd /k "docker exec -e TERM=debian -ti --privileged=true host bash  -c "echo -ne '\033]0;host\007'; bash"" & start cmd /k "docker exec -e TERM=debian -ti --privileged=true server bash  -c "echo -ne '\033]0;server\007'; bash"" & start cmd /k "docker exec -e TERM=debian -ti --privileged=true ospf2 bash  -c "echo -ne '\033]0;ospf2\007'; bash"" & start cmd /k "docker exec -e TERM=debian -ti --privileged=true br bash  -c "echo -ne '\033]0;br\007'; bash"" & start cmd /k "docker exec -e TERM=debian -ti --privileged=true rip1 bash  -c "echo -ne '\033]0;rip1\007'; bash"" & start cmd /k "docker exec -e TERM=debian -ti --privileged=true ospf1 bash  -c "echo -ne '\033]0;ospf1\007'; bash"" & start cmd /k "docker exec -e TERM=debian -ti --privileged=true ospf3 bash  -c "echo -ne '\033]0;ospf3\007'; bash"" 


In [83]:
print(open_bash_template.replace('{commands}', command_string))

start cmd /k "start cmd /k "docker exec -e TERM=debian -ti --privileged=true rip2 bash  -c "echo -ne '\033]0;rip2\007'; bash"" & start cmd /k "docker exec -e TERM=debian -ti --privileged=true host bash  -c "echo -ne '\033]0;host\007'; bash"" & start cmd /k "docker exec -e TERM=debian -ti --privileged=true server bash  -c "echo -ne '\033]0;server\007'; bash"" & start cmd /k "docker exec -e TERM=debian -ti --privileged=true ospf2 bash  -c "echo -ne '\033]0;ospf2\007'; bash"" & start cmd /k "docker exec -e TERM=debian -ti --privileged=true br bash  -c "echo -ne '\033]0;br\007'; bash"" & start cmd /k "docker exec -e TERM=debian -ti --privileged=true rip1 bash  -c "echo -ne '\033]0;rip1\007'; bash"" & start cmd /k "docker exec -e TERM=debian -ti --privileged=true ospf1 bash  -c "echo -ne '\033]0;ospf1\007'; bash"" & start cmd /k "docker exec -e TERM=debian -ti --privileged=true ospf3 bash  -c "echo -ne '\033]0;ospf3\007'; bash"" "


In [84]:
os.system(open_bash_template.replace('{commands}', command_string))

0